In [2]:
import logging
import urllib
from pathlib import Path

import requests
from config import Config
from requests_ntlm import HttpNtlmAuth

In [3]:
base_dir = Path(".").resolve()

In [9]:
class BookingData:
    """
    Represents booking data for generating SSRS reports.

    Parameters:
        - destination (str): The destination for the booking data.
        - date_from (str): The start date for the specified option.
        - date_to (str): The end date for the specified option.
        - option (int): The option to choose the type of date.

    Options:
        - 1: Arrival date
        - 2: In-house date
        - 3: Max processed date

    Usage:
        Specify the destination, date range, and option to create a BookingData instance.
    """

    def __init__(self, destination, option, date_from=None, date_to=None, ref_id=None):
        """
        Initialize a BookingData instance.

        Args:
            destination (str): The destination for the booking data.
            date_from (str): The start date for the specified option.
            date_to (str): The end date for the specified option.
            option (int): The option to choose the type of date.

        Raises:
            ValueError: If the option is not between 1 and 3.
        """
        self.ssrs_url = (
            Config.SSRS_BASE_URL + destination + " Reports/Reservations/Bookings Data"
        )
        self.ssrs_usr = Config.SSRS_USERNAME
        self.ssrs_pwd = Config.SSRS_PASSWORD

        if option == 1:
            self.payload = [
                ("from", date_from),
                ("to", date_to),
            ]
        elif option == 2:
            self.payload = [
                ("d1", date_from),
                ("d2", date_to),
            ]
        elif option == 3:
            self.payload = [
                ("MaxProcessDate_from", date_from),
                ("MaxProcessDate_to", date_to),
            ]
        elif option == 4:
            self.payload = [
                ("RefIDs", ref_id),
            ]
        else:
            raise ValueError("Option should be between 1 and 4.")

        self.payload.extend(
            [
                ("ReportParameter1", True),
                ("rs:ParameterLanguage", ""),
                ("rs:Command", "Render"),
                ("rs:Format", "CSV"),
                ("rc:ItemPath", "table1"),
            ]
        )

        self.params = urllib.parse.urlencode(self.payload, quote_via=urllib.parse.quote)

    def get(self):
        """
        Retrieve booking data from the SSRS server.

        Returns:
            str or None: The booking data as a string if available, or None if there's no new data.

        Raises:
            requests.exceptions.HTTPError: If an HTTP error occurs during the request.
            requests.exceptions.RequestException: If a general request error occurs.
        """
        logging.basicConfig(
            filename=base_dir / "logs" / "get_booking_data.log",
            level=logging.INFO,
            format="%(asctime)s - %(levelname)s: %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S",
        )

        try:
            response = requests.get(
                self.ssrs_url,
                params=self.params,
                stream=True,
                auth=HttpNtlmAuth(self.ssrs_usr, self.ssrs_pwd),
            )

            response.raise_for_status()

            return response.content.decode("utf8")

            # if len(data) > 424:
            #    logging.info(f"Request to {self.ssrs_url} was successful.")
            #    return data
            # else:
            #    logging.warning(f"No new data available for: {self.ssrs_url.split('?')[1]}")
        except requests.exceptions.HTTPError as e:
            logging.error(f"HTTP error occurred: {str(e)}")
            logging.exception("Full traceback:")
            raise  # re-raise the exception for the caller to handle if needed
        except requests.exceptions.RequestException as e:
            logging.error(f"An error occurred while making the request: {str(e)}")
            logging.exception("Full traceback:")
            raise  # re-raise the exception for the caller to handle if needed

        return None

In [18]:
from sklearn.pipeline import Pipeline
from transformers import BookingDataEncoder, BookingDataReadCsv

pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        ("data_encoder", BookingDataEncoder()),
    ]
)

booking_data = BookingData(
    destination="DU",
    ref_id="2169127,2169159",
    option=4,
).get()

df = pipeline.fit_transform(booking_data)
df.head()

,ref_id,res_id,hotel_id,operator_id,operator_code,bkg_ref,guest_name,sales_date,in_date,out_date,...,status4,status5,purchase_contract_id,purchase_spo_id,sales_contract_id,sales_spo_id,sales_spo_name,sales_spo_code,purchase_spo_name,purchase_spo_code
0,2169127,1851126,191620,1015,191620,501979914,BENITA1 ODOWD1,2024-02-02 00:00:00,2024-02-06 00:00:00,2024-02-10 00:00:00,...,Ame,Xml,307234,923202,307270,0,None,None,February Campaign Offer v2,FCO
1,2169159,1851155,191620,1015,191620,501981314,AMAR1 ALI1,2024-02-02 00:00:00,2024-02-04 00:00:00,2024-02-09 00:00:00,...,Ame,Xml,307234,923202,307270,0,None,None,February Campaign Offer v2,FCO


In [4]:
class ContractMonitor:
    def __init__(self, hotel_id, destination, season):
        self.ssrs_url = (
            Config.SSRS_BASE_URL + destination + " Reports/Contracting/Contract Monitor"
        )
        self.ssrs_usr = Config.SSRS_USERNAME
        self.ssrs_pwd = Config.SSRS_PASSWORD
        self.payload = [
            ("Seas:isnull", True),
            ("Season", season),
            ("City", "ALL"),
            ("Hotel_Chain", "ALL"),
            ("hotel", hotel_id),
            ("operator", 1000000),
            ("CreateDateFrom:isnull", True),
            ("CreateDateTo:isnull", True),
            ("contracttype", "A"),
            ("contracttype", "S"),
            ("contractstatus", "E"),
            # ("contractstatus", "H"),
            ("Revize", 2),
            ("showauth", True),
            ("view", 1),
            ("citycontract", 1),
            ("citycontract", 0),
            ("rs:ParameterLanguage", ""),
            ("rs:Command", "Render"),
            ("rs:Format", "CSV"),
            ("rc:ItemPath", "table1"),
        ]

        self.params = urllib.parse.urlencode(self.payload, quote_via=urllib.parse.quote)

    def get(self):
        response = requests.get(
            self.ssrs_url,
            params=self.params,
            stream=True,
            auth=HttpNtlmAuth(self.ssrs_usr, self.ssrs_pwd),
        )
        print(response.url, response.status_code)
        if response.status_code == 200:
            data = response.content.decode("utf8")
            return data
        return None

In [5]:
contract_monitor = ContractMonitor(1000000, "DU", "All 21-22")

In [6]:
data = contract_monitor.get()

http://95.211.42.206/ReportServer_SSRS?/DU%20Reports/Contracting/Contract%20Monitor&Seas%3Aisnull=True&Season=All%2021-22&City=ALL&Hotel_Chain=ALL&hotel=1000000&operator=1000000&CreateDateFrom%3Aisnull=True&CreateDateTo%3Aisnull=True&contracttype=A&contracttype=S&contractstatus=E&Revize=2&showauth=True&view=1&citycontract=1&citycontract=0&rs%3AParameterLanguage=&rs%3ACommand=Render&rs%3AFormat=CSV&rc%3AItemPath=table1 200


In [7]:
import io

import pandas as pd

contracts = pd.read_csv(
    io.StringIO(data),
    header=0,
    skiprows=2,
    names=[
        "contract_id",
        "contract_type",
        "status",
        "purchase_contract_id",
        "revise_number",
        "prev_contract_id",
        "is_revised",
        "operator_name",
        "hotel_id",
        "hotel_name_gwg",
        "main_season",
        "create_date",
        "begin_date",
        "end_date",
        "res_start_date",
        "res_end_date",
        "meal_plan",
        "history",
        "activation_date",
    ],
    usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 20, 21, 22, 23, 24, 25, 26, 35, 37],
    dtype={
        "contract_id": pd.Int64Dtype(),
        "contract_type": str,
        "status": str,
        "purchase_contract_id": pd.Int64Dtype(),
        "revise_number": pd.Int64Dtype(),
        "prev_contract_id": pd.Int64Dtype(),
        "is_revised": bool,
        "operator_name": object,
        "hotel_id": pd.Int64Dtype(),
        "hotel_name_gwg": str,
        "main_season": str,
        "meal_plan": str,
        "history": str,
    },
    parse_dates=[
        "create_date",
        "begin_date",
        "end_date",
        "res_start_date",
        "res_end_date",
        "activation_date",
    ],
)

contracts.sort_values(by=["hotel_name_gwg", "create_date"], inplace=True)

In [8]:
contracts.to_csv("contract_monitor_21_22.csv", index=False)

In [9]:
contracts.shape

(26228, 19)

In [1]:
class OperatorData:
    def __init__(self, destination):
        self.ssrs_url = Config.SSRS_BASE_URL + destination + " Active Tour Operators"
        self.ssrs_usr = Config.SSRS_USERNAME
        self.ssrs_pwd = Config.SSRS_PASSWORD
        self.payload = [
            ("rs:Command", "Render"),
            ("rs:Format", "CSV"),
            ("rc:ItemPath", "table1"),
        ]

        self.params = urllib.parse.urlencode(self.payload, quote_via=urllib.parse.quote)

    def get(self):
        response = requests.get(
            self.ssrs_url,
            params=self.params,
            stream=True,
            auth=HttpNtlmAuth(self.ssrs_usr, self.ssrs_pwd),
        )

        if response.status_code == 200:
            data = response.content.decode("utf8")
            return data
        return None